# GET Query Tests

GET query format, expected parameters, and example requests.  Pulls from live db_api.  This notebook doubles as a manual testing framework for major functionality of the database application and RDS instance it interacts with.

## 1. Biz Words

> Word cloud over time

> Package Format: {'schema': 'biz_words', 'params': {'business_id': biz_id_to_lookup}

In [7]:
import requests
import pandas as pd
import ujson
import numpy as np

url='https://db-api-yelp18-staging.herokuapp.com/api/data' # Current DB-API url for reference

In [5]:
%%time
# biz_words get request
package = {
    'schema': 'biz_words',
    'params': {
        'business_id': 'ajoqEHnCZTD8-8GqGLq9-Q'
        },
}
response = requests.get(url=url, json=package)
print('Status: ', response.status_code)
print('Content: ', response.text)

Status:  200
Content:  {
  "data": [
    [
      "Thu, 02 Jun 2005 05:09:34 GMT", 
      "{Palms,MTV,Real,World,fame,pretty,way,portrayed,young,drunk,fabulous,probably,smaller,hotels,wo,run,trade,convention,set,properties,rooms,pretty,decent,specialty,suites,aforementioned,Real,World,suite,left,pretty,way,rent,night,depending,weekend,bachelor,party,suite,complete,stripper,pole,foot,bed,think,goes,night,celebrity,stalker,place,fail,famous,usually,Ghost,Bar,located,floor,giving,killer,view,strip,Hart,Huntington,tattoo,parlor,subject,new,A&E,reality}", 
      4.0
    ], 
    [
      "Thu, 23 Jun 2005 23:57:39 GMT", 
      "{Rain,nightclub,palms,great,place,dance,meet,young,beautiful,people}", 
      4.0
    ], 
    [
      "Fri, 11 Nov 2005 00:08:55 GMT", 
      "{Rain,Ghost,Bar,hype,hotties,guess,people,afraid,heights,know,Vegas,drunk,drunk,look,feet,think,far,fall,Thank,god,making,king,size,beds,obnoxiously,loud,patterns,hit,matter,bad,aim}", 
      4.0
    ], 
    [
      "Tue, 10 Jan 

**Running** the same request via pgAdmin directly to the database from local connection shows query time at: 2.3s

> Query: SELECT date, token, stars
FROM reviews
WHERE business_id = 'ajoqEHnCZTD8-8GqGLq9-Q'
ORDER BY date;

* Deployed application may have significant latency improvements over local program

In [3]:
%%time
# Testing upstream parsing

def get_reviews(business_id, url='https://db-api-yelp18-staging.herokuapp.com/api/data'):
    """Create JSON request object and send GET request to database api
    """
    package = {
    'schema': 'biz_words',
    'params': {
        'business_id': business_id
        },
    }
    response = requests.get(url=url, json=package)
    return strip_tokens_badchar(
        pd.DataFrame(ujson.loads(response.text)['data'], columns=['date', 'tokens', 'star_review'])
    )

# Custom Cleaning function for current token information.
# Remove for performance gain in future data release
def strip_tokens_badchar(dataframe):
    # Clean string with simple regex
    dataframe['tokens'] = dataframe['tokens'].str.strip('\{').str.strip('\}').\
        str.split(',')

    return dataframe

response = get_reviews(business_id='ajoqEHnCZTD8-8GqGLq9-Q')
response

CPU times: user 37.3 ms, sys: 5.84 ms, total: 43.1 ms
Wall time: 1.7 s


,date,tokens,star_review
0,"Thu, 02 Jun 2005 05:09:34 GMT","[Palms, MTV, Real, World, fame, pretty, way, p...",4.0
1,"Thu, 23 Jun 2005 23:57:39 GMT","[Rain, nightclub, palms, great, place, dance, ...",4.0
2,"Fri, 11 Nov 2005 00:08:55 GMT","[Rain, Ghost, Bar, hype, hotties, guess, peopl...",4.0
3,"Tue, 10 Jan 2006 07:34:06 GMT","[Yes, Palms, featured, MTV, Real, World, come,...",5.0
4,"Wed, 05 Apr 2006 23:55:22 GMT","[Palm, favorite, time, customer, service, abso...",4.0
...,...,...,...
1420,"Sun, 28 Oct 2018 00:08:47 GMT","[spa, great, wife, birthday, gift, room, lot, ...",3.0
1421,"Tue, 30 Oct 2018 22:12:04 GMT","[hotel, currently, construction, great, distur...",1.0
1422,"Thu, 08 Nov 2018 07:50:42 GMT","[Sucks, s, renovation, staffs, rude, awful, da...",1.0
1423,"Sat, 10 Nov 2018 21:40:10 GMT","[terrible, terrible, terrible, stayed, nights,...",1.0


In [158]:
def wc_count(agg_row):
    """Count the occurance of each word and rank
    """
    docs = agg_row[0]
    total=len(docs)
    wc = pd.DataFrame({'word':docs, 'count':np.ones(total)})
    wc = wc.groupby('word').sum()
    wc['pct_total'] = wc['count']/total
    wc['rank'] = wc['count'].rank(method='first', ascending=False)
    wc['date'] = agg_row[2]
    wc['star_review'] = agg_row[1]
    return wc.sort_values(by='rank').nlargest(30, 'count')


def timeseries(bus_id):
    # Deprecated - See db_api get request functionality
    # result = reviews.query.with_entities(reviews.tokens, reviews.date, \
    #          reviews.star_review).filter_by(business_id=bus_id)
    # df = pd.read_sql(sql = result.statement, con = DB.engine)
    # df['tokens'] = df['tokens'].str.strip('\[').str.strip('\]').\
    #     str.split(', ')
    # filtered = df.sort_values('date')
    
    # Send API request and process response to DataFrame
    filtered = get_reviews(business_id=bus_id)
    
    # Group processed response
    filtered['bins'] = pd.qcut(filtered.index, q=10, precision=0)
    new_df = filtered.groupby('bins').agg({'tokens': 'sum', \
            'star_review': 'mean', 'date': lambda x: x.iloc[-1]})

    # Get word counts
    counts = list(map(wc_count, new_df.to_numpy()))
    
    # Generate Ouput BETA
    output = list(map(lambda x: {x.date[0]:x.reset_index().to_dict('r')}, counts))

    # Generate output <- This is causing the problem.  The dates end up out of order for some reason
#     df_final = pd.concat(counts).reset_index()
#     output = (df_final.groupby(['date'], as_index=True)
#              .apply(lambda x: x[['word','count','pct_total','rank',\
#                  'star_review']].to_dict('r'))
#               .to_json()).replace("'", "")

    return ujson.dumps(output)

In [159]:
%%time
output = timeseries('ajoqEHnCZTD8-8GqGLq9-Q')

CPU times: user 185 ms, sys: 4.33 ms, total: 190 ms
Wall time: 1.71 s


In [160]:
output

'[{"Tue, 02 Mar 2010 00:18:40 GMT":[{"word":"Palms","count":164.0,"pct_total":0.0147707827,"rank":1.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"room","count":156.0,"pct_total":0.0140502567,"rank":2.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"place","count":107.0,"pct_total":0.009637035,"rank":3.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"night","count":90.0,"pct_total":0.0081059173,"rank":4.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"nice","count":89.0,"pct_total":0.0080158516,"rank":5.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"stay","count":87.0,"pct_total":0.0078357201,"rank":6.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"time","count":85.0,"pct_total":0.0076555886,"rank":7.0,"date":"Tue, 02 Mar 2010 00:18:40 GMT","star_review":3.4825174825},{"word":"hotel","count":83.0,"pct_total":0

## Vizualization 2 Get_Query development

SQLAlchemy db request and any expansion or early formatting prior to send.  The data is stored in PostgreSQL nested and jsonified to prevent posting errors and contains escape characters, other artifacts, that should be removed before client recieves data.

In [141]:
import requests
import pandas as pd
import ujson
import numpy as np

local_url='http://127.0.0.1:5050/api/data' # Current DB-API url for reference or local
live_url='https://db-api-yelp18-staging.herokuapp.com/api/data'

In [142]:
%%time
# biz_comp get request
package = {
    'schema': 'biz_comp',
    'params': {
        'business_id': 'ajoqEHnCZTD8-8GqGLq9-Q'
        },
}
response = requests.get(url=live_url, json=package)
print('Status: ', response.status_code)
print('Content: ', response.text)

Status:  200
Content:  {
  "address": "4321 W Flamingo Rd", 
  "avg_stars": [
    3.4825174825174825, 
    3.2464788732394365, 
    3.3076923076923075, 
    3.084507042253521, 
    3.125874125874126, 
    2.852112676056338, 
    2.4895104895104896, 
    2.73943661971831, 
    2.6783216783216783, 
    2.471830985915493
  ], 
  "bestinsector": [
    "ARIA Resort & Casino", 
    "The Cosmopolitan of Las Vegas", 
    "McCarran International Airport", 
    "The Buffet", 
    "The Venetian Las Vegas"
  ], 
  "business_id": "ajoqEHnCZTD8-8GqGLq9-Q", 
  "categories": "Arts & Entertainment, Hotels & Travel, Event Planning & Services, Hotels, Resorts, Restaurants, Casinos", 
  "chunk_sentiment": [
    1.0, 
    1.0, 
    1.0, 
    0.8, 
    0.8, 
    0.8, 
    0.7707031249999999, 
    0.7707031249999999, 
    0.7410714285714286, 
    0.7410714285714286, 
    -0.6145833333333334, 
    -0.625, 
    -0.625, 
    -0.6333333333333334, 
    -0.64, 
    -0.6435185185185186, 
    -0.66875, 
    -0.66875

In [145]:
ujson.loads(response.text)

{'address': '4321 W Flamingo Rd',
 'avg_stars': [3.482517482517482,
  3.246478873239436,
  3.307692307692307,
  3.084507042253521,
  3.125874125874126,
  2.852112676056338,
  2.489510489510489,
  2.73943661971831,
  2.678321678321678,
  2.471830985915493],
 'bestinsector': ['ARIA Resort & Casino',
  'The Cosmopolitan of Las Vegas',
  'McCarran International Airport',
  'The Buffet',
  'The Venetian Las Vegas'],
 'business_id': 'ajoqEHnCZTD8-8GqGLq9-Q',
 'categories': 'Arts & Entertainment, Hotels & Travel, Event Planning & Services, Hotels, Resorts, Restaurants, Casinos',
 'chunk_sentiment': [1.0,
  1.0,
  1.0,
  0.8,
  0.8,
  0.8,
  0.770703124999999,
  0.770703124999999,
  0.741071428571428,
  0.741071428571428,
  -0.614583333333333,
  -0.625,
  -0.625,
  -0.6333333333333331,
  -0.64,
  -0.643518518518518,
  -0.6687500000000001,
  -0.6687500000000001,
  -0.866666666666666,
  -1.0],
 'city': 'Las Vegas',
 'competitors': ['Staybridge Suites Las Vegas',
  'A Robert Cromeans Salon',
  'W

In [59]:
import re
components = re.findall(r'\[([^]]+)\]', test)
components[0]

'3.4825174825174825, 3.2464788732394365, 3.3076923076923075, 3.084507042253521, 3.125874125874126, 2.852112676056338, 2.4895104895104896, 2.73943661971831, 2.6783216783216783, 2.471830985915493'